In [9]:
from utils import list_images, read_image, extract_pixel_size, correct_pixel_size, make_isotropic, remove_labels_touching_edge, simulate_cytoplasm
from pathlib import Path
import napari
import pyclesperanto_prototype as cle
from skimage.measure import regionprops_table
import pandas as pd

cle.select_device("RTX")

<NVIDIA GeForce RTX 4090 Laptop GPU on Platform: NVIDIA CUDA (2 refs)>

In [2]:
# Copy the path where your images are stored, you can use absolute or relative paths to point at other disk locations
directory_path = Path("./raw_data/condition_1")

# Define the channels you want to analyze using the following structure:
# markers = [(channel_name, channel_nr),(..., ...)]
# Remember in Python one starts counting from 0, so your first channel will be 0
# i.e. markers = [("c3neo", 0), ("myc-tag", 1)]

markers = [("c3neo", 0), ("myc-tag", 1)]
nuclei_channel = 0

# Image size reduction (downsampling) to improve processing times (slicing, not lossless compression)
# Now, in addition to xy, you can downsample across your z-stack
slicing_factor_xy = 4 # Use 2 or 4 for downsampling in xy (None for lossless)
slicing_factor_z = None # Use 2 to select 1 out of every 2 z-slices

# Iterate through the .czi and .nd2 files in the raw_data directory
images = list_images(directory_path)

images

['raw_data\\condition_1\\01.nd2']

In [3]:
image = images[0]

img, filename = read_image(image, slicing_factor_xy, slicing_factor_z)

# Obtain xyz pixel (voxel) size from image metadata (nd2 files)
voxel_size = extract_pixel_size(images[0])

# Correct pixel size if slicing is applied
pixel_size_x, pixel_size_y, voxel_size_z = correct_pixel_size(voxel_size, slicing_factor_xy, slicing_factor_z)



Image analyzed: 01
Original Array shape: (3, 31, 2720, 2720)
Compressed Array shape: (3, 31, 680, 680)
Pixel size: 0.065 µm x 0.065 µm
Voxel (Z-step) size: 0.200 µm


In [ ]:
nuclei_resampled = make_isotropic(img[nuclei_channel], pixel_size_x, pixel_size_y, voxel_size_z)

# Remove background with a top_hat_filter
#background_subtracted = cle.top_hat_box(nuclei_resampled, radius_x=5, radius_y=5, radius_z=5)

# Apply gaussian blur to prevent the formation of holes upon labeling
post_gaussian = cle.gaussian_blur(nuclei_resampled, sigma_x=2, sigma_y=2, sigma_z=2)

# Voronoi-Otsu labeling
segmented = cle.voronoi_otsu_labeling(post_gaussian, spot_sigma=10, outline_sigma=1)

# Close holes in labels to avoid false emtpy volumes within the nuclei
segmented = cle.closing_labels(segmented, radius=5)

# Remove labels touching image xy borders
nuclei_labels = remove_labels_touching_edge(segmented.get())

# Simulate cytoplasm
cytoplasm_labels = simulate_cytoplasm(nuclei_labels, dilation_radius=10)

In [5]:
viewer = napari.Viewer(ndisplay=2)

viewer.add_image(nuclei_resampled)
viewer.add_labels(nuclei_labels)
viewer.add_labels(cytoplasm_labels)

<Labels layer 'cytoplasm_labels' at 0x17ae1ee2920>

In [ ]:
# Create an  empty list to hold the per channel info
props_list = []

# Create a dictionary containing all image descriptors
descriptor_dict = {"filename": filename, "folder_name": "PLACEHOLDER"}

# Loop through each channel, resample it and extract the average intensity within nuclei and cytoplasm
for channel_name, ch_nr in markers:
    print(f"Analyzing channel: {channel_name}")

    # Resample marker and transform into numpy array
    marker_resampled = make_isotropic(img[ch_nr], pixel_size_x, pixel_size_y, voxel_size_z).get()

    # Extract intensity information from each marker channel
    nuclei_props = regionprops_table(label_image=nuclei_labels,
                            intensity_image=marker_resampled,
                            properties=["label", "intensity_mean"])
    
    # Extract intensity information from each marker channel
    cyto_props = regionprops_table(label_image=cytoplasm_labels,
                            intensity_image=marker_resampled,
                            properties=["label", "intensity_mean"])
    
    # Convert to dataframe
    nuclei_props_df = pd.DataFrame(nuclei_props)
    cyto_props_df = pd.DataFrame(cyto_props)

    # Rename intensity_mean column to indicate the specific image
    nuclei_props_df.rename(columns={"intensity_mean": f"nuclei_{channel_name}_avg_int"}, inplace=True)
    cyto_props_df.rename(columns={"intensity_mean": f"cyto_{channel_name}_avg_int"}, inplace=True)

    # Merge nuclei and cyto on label
    props_df = pd.merge(nuclei_props_df, cyto_props_df, on='label')

    # Append each props_df to props_list
    props_list.append(props_df)

# Initialize the df with the first df in the list
img_props_df = props_list[0]
# Start looping from the second df in the list
for df in props_list[1:]:
    img_props_df = img_props_df.merge(df, on="label")

# Add each key-value pair from descriptor_dict to props_df at the specified position
insertion_position = 0
for key, value in descriptor_dict.items():
    img_props_df.insert(insertion_position, key, value)
    insertion_position += 1  # Increment position to maintain the order of keys in descriptor_dict

Analyzing channel: c3neo
Analyzing channel: myc-tag


In [19]:
img_props_df

,filename,folder_name,label,nuclei_c3neo_avg_int,cyto_c3neo_avg_int,nuclei_myc-tag_avg_int,cyto_myc-tag_avg_int
0,01,PLACEHOLDER,1,1311.049072,1112.524292,1301.950317,1367.415039
1,01,PLACEHOLDER,2,1314.661011,1085.760010,1100.068481,1118.131836
2,01,PLACEHOLDER,3,1394.914673,1165.139893,1176.893921,1213.254272
3,01,PLACEHOLDER,4,1382.904907,1156.552246,1189.783325,1260.628052
4,01,PLACEHOLDER,5,1408.171631,1126.957275,1313.267456,1461.038086
5,01,PLACEHOLDER,6,1460.184814,1097.128296,1264.121826,1316.127197
6,01,PLACEHOLDER,7,1321.119507,1082.198364,1122.595459,1176.105835
7,01,PLACEHOLDER,8,1328.834351,1096.716919,1162.748657,1212.943726
8,01,PLACEHOLDER,9,1337.351074,1068.161743,1230.003052,1337.420776
9,01,PLACEHOLDER,10,1332.568481,1081.269531,1149.766479,1209.043213


In [ ]:
#TODO: Extract folder_names, loop over them, then loop over each image inside them. Save .csv on a per folder name basis.